In [2]:
import pickle as pkl
import numpy as np
import os
# os.chdir('/scratch/cl1205/protease-gcnn-pytorch/model')
# from utils import *
import torch
from torch import nn
import pandas as pd

In [3]:
# other metrics summary
def getMetrics(path, dataList, split=2): # modified based on getCrossTestMetrics
    results = []
    # logits_test_HCV_WT_binary_10_ang_aa_energy_7_energyedge_5_hbond_model_model_for_test_A171T_seq_energy
    results.append(['data', 'feature','accuracy', 'auc', 'precision', 'recall', 'f1', 'ap'])
    for data in dataList:
        y = pkl.load(open('/scratch/cl1205/protease-gcnn-pytorch/data/ind.' + data[0] + '_binary_10_ang_aa_energy_7_energyedge_5_hbond.y', 'rb'))
        if split == 2:
            maskPath = '/scratch/cl1205/protease-gcnn-pytorch/data/ind.' + data[0] + '_binary_10_ang_aa_energy_7_energyedge_5_hbond.test.index'
        elif split == 3:
            maskPath = '/scratch/cl1205/protease-gcnn-pytorch/data/ind.' + data[0] + '_binary_10_ang_aa_energy_7_energyedge_5_hbond.trisplit.test.index'

        mask = np.loadtxt(maskPath, dtype=int)
        mask = np.sort(mask)
        #adj_ls, features, labels, sequences, proteases, labelorder, train_mask, test_mask = load_data(data + '_binary_10_ang_aa_energy_7_energyedge_5_hbond', is_test=data + '_binary_10_ang_aa_energy_7_energyedge_5_hbond', norm_type=False, test_format = 'split', energy_only=False, noenergy=False)
        if data == 'WT' or data == 'all': #Just for HCV
            y = y[mask[:-1],:]
        else:
            y = y[mask,:]
#         y = y[mask, :]
        print(data)
#         y = labels[test_mask]
        logits_test = pkl.load(open(data[1], 'rb'))
        feature = data[1].split('/')[-2]
        acc = getAcc(logits_test, np.argmax(y,axis=1))
        precision, recall, f1, ap = precision_recall(np.argmax(y,axis=1), np.argmax(logits_test.cpu().detach().numpy(),axis=1))
        #proba = softmax(logits_test.numpy(),1)
        proba = nn.functional.softmax(logits_test)
        auc = float(getAUC(np.argmax(y,axis=1), proba))
        results.append([data[0], feature, acc, auc, precision, recall, f1, ap])
    results = pd.DataFrame(results[1:], columns = results[0])
    return results

In [4]:
from sklearn.metrics import roc_curve, auc, average_precision_score
def getAcc(output, labels):
    preds = np.argmax(output.cpu().detach().numpy(),axis=1)
    correct = np.sum(preds == labels)
#    correct = np.sum(preds == labels)
    #correct = correct.sum()
#     return correct
    #print(correct / len(labels))
    return correct / len(labels)
def getAUC(true, proba):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # cleaved = 0
    fpr, tpr, _ = roc_curve(true, proba[:,0], pos_label=0)
    roc_auc = auc(fpr, tpr) #only for binary class
    return '{:.4f}'.format(roc_auc)
def precision_recall(true, predict, pos_label=0):
    # true positive, false negative, false positive
    tp = np.sum(true[np.where(predict == pos_label)] == pos_label)
    fp = np.sum(true[np.where(predict == pos_label)] != pos_label)
    fn = np.sum(true[np.where(predict != pos_label)] == pos_label)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    ap = average_precision_score(true, predict)
    return precision, recall, f1, ap
def softmax(x,n):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x)
    if n == 1:
        return e_x / e_x.sum(axis=n).reshape(-1,1)
    else:
        return e_x / e_x.sum(axis=n)

#### triple split formal results

In [50]:
data2List = []
paths = ['/projects/f_sdk94_1/PGCN/TEV/trisplit/models', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/WT',
         '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T','/projects/f_sdk94_1/PGCN/HCV/trisplit/models/D183A',
         '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined']
dataname = ['TEV_all', 'HCV_WT', 'HCV_A171T', 'HCV_D183A', 'HCV_Triple', 'HCV_all']
for i in range(len(dataname)):
    for root, subdirs, files in os.walk(paths[i]):
        for file in files:
            if file.split('_')[0] == 'logits':
                data2 = os.path.join(root, file)
                data2List.append([dataname[i], data2])
results = getMetrics(os.path.join(root,file), data2List, split=3)

['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/seq/logits_test_seed_2_hidden_20_linear_0_lr_0.005_wd_0.0001_bs_100_dt_0.1']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/seq/logits_test_seed_1_hidden_20_linear_0_lr_0.0005_wd_0.0005_bs_50_dt_0.2']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/seq/logits_test_seed_3_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_10_dt_0.05']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/energy/logits_test_seed_2_hidden_20_linear_0_lr_0.001_wd_0.001_bs_50_dt_0.1']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/energy/logits_test_seed_3_hidden_20_linear_0_lr_0.01_wd_0.0005_bs_500_dt_0.01']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/energy/logits_test_seed_1_hidden_20_linear_0_lr_0.005_wd_0.005_bs_1000_dt_0.1']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/trisplit/models/complete/logits_test_seed_2_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_10_dt_0.2']
['TEV_all', '/projects/f_sdk94_1/PGCN/TEV/

/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ip

['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/seq/logits_test_seed_1_hidden_20_linear_0_lr_0.001_wd_0.0005_bs_100_dt_0.4']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/energy/logits_test_seed_2_hidden_20_linear_0_lr_0.01_wd_0.001_bs_1000_dt_0.2']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/energy/logits_test_seed_3_hidden_20_linear_0_lr_0.01_wd_0.001_bs_1000_dt_0.4']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/energy/logits_test_seed_1_hidden_20_linear_0_lr_0.005_wd_0.0005_bs_50_dt_0.1']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/complete/logits_test_seed_2_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_50_dt_0.1']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/complete/logits_test_seed_1_hidden_20_linear_0_lr_0.001_wd_0.0005_bs_100_dt_0.3']
['HCV_A171T', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/A171T/complete/logits_test_seed_3_hidden_20_linear_0_lr_0.0005_wd_

/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ip


['HCV_D183A', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/D183A/complete/logits_test_seed_2_hidden_20_linear_0_lr_0.0005_wd_0.0005_bs_500_dt_0.4']
['HCV_D183A', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/D183A/complete/logits_test_seed_1_hidden_20_linear_0_lr_0.05_wd_0.0005_bs_500_dt_0.01']
['HCV_Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple/seq/logits_test_seed_1_hidden_20_linear_0_lr_0.005_wd_0.0001_bs_50_dt_0.5']
['HCV_Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple/seq/logits_test_seed_3_hidden_20_linear_0_lr_0.005_wd_0.0001_bs_100_dt_0.2']
['HCV_Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple/seq/logits_test_seed_2_hidden_20_linear_0_lr_0.001_wd_0.001_bs_10_dt_0.01']
['HCV_Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple/energy/logits_test_seed_1_hidden_20_linear_0_lr_0.005_wd_0.0001_bs_50_dt_0.05']
['HCV_Triple', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Triple/energy/logits_test_seed_2_hidden_20_linear_0_lr_0.000

/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ip

['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/seq/logits_test_seed_3_hidden_20_linear_0_lr_0.001_wd_0.0001_bs_100_dt_0.1']
['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/energy/logits_test_seed_2_hidden_20_linear_0_lr_0.005_wd_0.0005_bs_500_dt_0.01']
['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/energy/logits_test_seed_1_hidden_20_linear_0_lr_0.01_wd_0.0005_bs_500_dt_0.01']
['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/energy/logits_test_seed_3_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_10_dt_0.05']
['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/complete/logits_test_seed_2_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_50_dt_0.4']
['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/complete/logits_test_seed_3_hidden_20_linear_0_lr_0.0005_wd_0.0001_bs_10_dt_0.5']


/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ip

['HCV_all', '/projects/f_sdk94_1/PGCN/HCV/trisplit/models/Combined/complete/logits_test_seed_1_hidden_20_linear_0_lr_0.001_wd_0.0001_bs_1000_dt_0.1']


/projects/community/py-data-science-stack/5.1.0/kp807/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [51]:
results.to_csv('MetricSummary_PGCN_AUC_Precision_Recall_F1_AP.csv',index=None)

In [52]:
table = pd.pivot_table(results, values=['accuracy', 'auc','precision','recall','f1','ap'], index=['data'],
                    columns=['feature'], aggfunc=np.mean)

In [53]:
table

accuracy                            ap                      \
feature     complete    energy       seq  complete    energy       seq   
data                                                                     
HCV_A171T   0.964169  0.957860  0.964673  0.960966  0.955326  0.960484   
HCV_D183A   0.899747  0.890761  0.900309  0.913671  0.912018  0.919002   
HCV_Triple  0.934698  0.924464  0.938109  0.906608  0.892577  0.912850   
HCV_WT      0.921088  0.901587  0.919274  0.934556  0.917855  0.934306   
HCV_all     0.923142  0.910318  0.923567  0.924972  0.912938  0.923649   
TEV_all     0.877225  0.866176  0.868631  0.861837  0.855896  0.858459   

                 auc                            f1                      \
feature     complete    energy       seq  complete    energy       seq   
data                                                                     
HCV_A171T   0.978700  0.976100  0.979000  0.934625  0.923006  0.935304   
HCV_D183A   0.953300  0.945500  0.955033  0.851307  0.841781  0.854776   
HCV_Triple  0.971000  0.965500  0.975233  0.929901  0.918620  0.933959   
HCV_WT      0.955900  0.937900  0.954733  0.847266  0.808156  0.844595   
HCV_all     0.967800  0.960933  0.968933  0.887173  0.868222  0.887100   
TEV_all     0.940833  0.926100  0.940833  0.863203  0.853603  0.856352   

           precision                        recall                      
feature     complete    energy       seq  complete    energy       seq  
data                                                                    
HCV_A171T   0.963921  0.953343  0.968488  0.907060  0.894549  0.904379  
HCV_D183A   0.830170  0.803030  0.819651  0.874359  0.884615  0.893162  
HCV_Triple  0.945991  0.937923  0.944311  0.914609  0.900206  0.923868  
HCV_WT      0.852391  0.819434  0.845009  0.842932  0.797557  0.844677  
HCV_all     0.887957  0.869859  0.893449  0.886398  0.866716  0.880917  
TEV_all     0.843837  0.820754  0.822971  0.883754  0.890756  0.893557

In [54]:
table.to_csv('MetricAverageOverSeedSummary.csv',index=None)

# plot AUC or AP

In [ ]:
# plot AUC for combined features
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
def plot_AUC_AP(feature='all',metric='auc'):
    score_df = pd.read_csv("final_model_PGCN_parameters_scores_summary.csv",index_col=0)
    if feature == 'all':
        info = score_df.loc[:,score_df.loc['feature_encoding'] == 'Seq+Energy']
    elif feature == 'energy':
        info = score_df.loc[:,score_df.loc['feature_encoding'] == 'E-only']
    if metric == 'auc':
        fig = plt.figure(figsize=(8,8))
    elif metric == 'ap':
        _, ax = plt.subplots(figsize=(8, 8))
    lw = 2
    for i in info.columns:
        data_name = info.loc['Data', i]
        true = read_true_test_label(data_name)
        logits = pkl.load(open("model/outputs/tt_finalize_20210413/HCV_" + data_name + "_binary_10_ang_aa_energy_7_energyedge_5_hbond/bs_" + str((info.loc['batch_size',i])) + "/logits_test_seed_" + str(info.loc['seed',i]) + "_hidden_20_linear_0_lr_" + str(float(info.loc['learning_rate',i])) + "_wd_" + str(float(info.loc['weight_decay',i])) + "_bs_" + str(info.loc['batch_size',i]) + "_dt_" + str(float(info.loc['dropout',i])),'rb'))
        print("model/outputs/tt_finalize_20210413/HCV_" + data_name + "_binary_10_ang_aa_energy_7_energyedge_5_hbond/bs_" + str((info.loc['batch_size',i])) + "/logits_test_seed_" + str(info.loc['seed',i]) + "_hidden_20_linear_0_lr_" + str(float(info.loc['learning_rate',i])) + "_wd_" + str(float(info.loc['weight_decay',i])) + "_bs_" + str(info.loc['batch_size',i]) + "_dt_" + str(float(info.loc['dropout',i])))
        #if feature == 'E-only':
    #    logits = pkl.load(open("model/outputs/tt_finalize_20210413/HCV_" + data_name + "_binary_10_ang_energy_7_energyedge_5_hbond_rerun/bs_" + str((info['batch_size'])) + "/logits_test_seed_" + str(info['seed']) + "_hidden_20_linear_0_lr_" + str(float(info['learning_rate'])) + "_wd_" + str(float(info['weight_decay'])) + "_bs_" + str(info['batch_size']) + "_dt_" + str(float(info['dropout'])),'rb'))
    #elif feature == 'Seq+Energy':

        predict = np.argmax(logits.numpy(),axis=1)
        proba = softmax(logits.numpy(),1)

        if data_name == 'all':
            data_name = 'Combined'
            
        if metric == 'auc':
            # auc
            # cleaved = 0
            fpr, tpr, _ = roc_curve(true, proba[:,0], pos_label=0)
            roc_auc = auc(fpr, tpr) #only for binary class
            #print(roc_auc)

            plt.plot(
                fpr,
                tpr,
            #     color="darkorange",
                lw=lw,
                label="%s (AUC = %0.2f)" % (data_name, roc_auc),
            )
        elif metric == 'ap':
            # ap
            precision, recall, _ = precision_recall_curve(np.array(true==0, dtype=int), proba[:,0])
            _,_,f1, ap = precision_recall(true, predict, pos_label=0)
            
            display = PrecisionRecallDisplay(
            recall=recall,
            precision=precision,
            average_precision=ap,
            )
            display.plot(ax=ax, name=f"{data_name}")
            
#             f_scores = np.linspace(0.2, 0.8, num=4)
#             lines, labels = [], []
#             for f_score in f_scores:
#                 x = np.linspace(0.01, 1)
#                 y = f_score * x / (2 * x - f_score)
#                 (l,) = plt.plot(x[y >= 0], y[y >= 0], color="gray", alpha=0.2)
#                 plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

    if metric == 'auc':
        plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend(loc="lower right")
        fig.tight_layout()
    elif metric == 'ap':
#         handles, labels = display.ax_.get_legend_handles_labels()
#         handles.extend([l])
#         labels.extend(["iso-f1 curves"])
        # set the legend and the axes
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        #ax.legend(handles=handles, labels=labels, loc="best")
        plt.tight_layout()
    #plt.title("Receiver operating characteristic example")
    plt.savefig(metric + '_' + feature + '.png', format='png', dpi=300)
    plt.show()

In [ ]:
plot_AUC_AP(metric='auc',feature='energy')

# Summarize all models' accuracy and all other metrics

In [5]:
df_pgcn = pd.read_csv('../PGCN/outputs/MetricPerSeedSummary_PGCN_AUC_Precision_Recall_F1_AP.csv')
df_pgcn.head()

,data,feature,accuracy,auc,precision,recall,f1,ap
0,TEV_all,seq,0.869245,0.9408,0.827451,0.886555,0.855984,0.856544
1,TEV_all,seq,0.865562,0.9400,0.800000,0.924370,0.857700,0.865909
2,TEV_all,seq,0.871087,0.9417,0.841463,0.869748,0.855372,0.852924
3,TEV_all,energy,0.863720,0.9298,0.792857,0.932773,0.857143,0.867382
4,TEV_all,energy,0.865562,0.9304,0.836735,0.861345,0.848861,0.846302


In [6]:
df_pgcn['model'] = df_pgcn.shape[0] * ['PGCN']

In [4]:
df_ml = pd.read_csv('../PGCN/outputs/MetricSummary_ML_AUC_Precision_Recall_F1_AP.csv')
df_ml.head()

,data,model,feature,accuracy,auc,precision,recall,f1,ap
0,TEV_all,logistic_regression,complete,0.823204,0.902259,0.819820,0.764706,0.791304,0.790942
1,TEV_all,random_forest,complete,0.697974,0.773102,0.707865,0.529412,0.605769,0.670738
2,TEV_all,decision_tree,complete,0.644567,0.623151,0.633136,0.449580,0.525799,0.631836
3,TEV_all,svm,complete,0.802947,0.891473,0.719064,0.903361,0.800745,0.810985
4,TEV_all,ann,complete,0.806630,0.918446,0.709779,0.945378,0.810811,0.827619


In [5]:
data = ['HCV_WT','HCV_A171T','HCV_D183A','HCV_Triple','HCV_Combined','TEV_Combined']
df = pd.concat((df_ml, df_pgcn))
model = ['logistic_regression', 'random_forest','decision_tree','svm','ann', 'PGCN']
df_pivot = pd.pivot_table(df, values=['accuracy', 'auc','precision','recall','f1','ap'], index=['data','model'],
                    columns=['feature'], aggfunc=np.mean)
df_pivot = df_pivot.rename(index={'HCV_all': 'HCV_Combined', 'TEV_all': 'TEV_Combined'})
df_pivot = df_pivot.loc[[(x, y) for x in data for y in model],:].round(4)
df_pivot

accuracy                       ap          \
feature                          complete  energy     seq complete  energy   
data         model                                                           
HCV_WT       logistic_regression   0.8612  0.8340  0.8871   0.8528  0.8242   
             random_forest         0.8435  0.8449  0.9116   0.8609  0.8710   
             decision_tree         0.8068  0.8109  0.8735   0.8335  0.8375   
             svm                   0.8980  0.8667  0.9143   0.9140  0.8925   
             ann                   0.8925  0.8762  0.9102   0.9034  0.8895   
             PGCN                  0.9211  0.9016  0.9193   0.9346  0.9179   
HCV_A171T    logistic_regression   0.8085  0.8857  0.9569   0.9172  0.9411   
             random_forest         0.8887  0.7767  0.9493   0.9124  0.8874   
             decision_tree         0.7366  0.7139  0.9334   0.8201  0.7989   
             svm                   0.9500  0.9394  0.9591   0.9518  0.9384   
             ann                   0.9531  0.9145  0.9561   0.9544  0.9482   
             PGCN                  0.9642  0.9579  0.9647   0.9610  0.9553   
HCV_D183A    logistic_regression   0.8618  0.8745  0.8863   0.9132  0.8926   
             random_forest         0.7852  0.7767  0.8703   0.8431  0.8177   
             decision_tree         0.6714  0.6967  0.8416   0.7450  0.7546   
             svm                   0.8762  0.8677  0.8812   0.8863  0.8767   
             ann                   0.8812  0.8728  0.8812   0.8958  0.8779   
             PGCN                  0.8997  0.8908  0.9003   0.9137  0.9120   
HCV_Triple   logistic_regression   0.8553  0.7602  0.9240   0.7886  0.6870   
             random_forest         0.8260  0.8304  0.9181   0.7979  0.8098   
             decision_tree         0.7339  0.6594  0.9006   0.6859  0.6236   
             svm                   0.9196  0.8991  0.9269   0.8906  0.8704   
             ann                   0.9020  0.8918  0.9313   0.8531  0.8434   
             PGCN                  0.9347  0.9245  0.9381   0.9066  0.8926   
HCV_Combined logistic_regression   0.9006  0.8854  0.9057   0.8910  0.8687   
             random_forest         0.8380  0.8349  0.9116   0.8498  0.8493   
             decision_tree         0.7503  0.7789  0.8785   0.8014  0.8178   
             svm                   0.9075  0.8912  0.9218   0.9120  0.8926   
             ann                   0.9113  0.8922  0.9169   0.9125  0.8878   
             PGCN                  0.9231  0.9103  0.9236   0.9250  0.9129   
TEV_Combined logistic_regression   0.8232  0.8085  0.8177   0.7909  0.8152   
             random_forest         0.6980  0.7311  0.8613   0.6707  0.7028   
             decision_tree         0.6446  0.6906  0.8527   0.6318  0.6694   
             svm                   0.8029  0.8029  0.8435   0.8110  0.8176   
             ann                   0.8066  0.8011  0.8361   0.8276  0.8162   
             PGCN                  0.8772  0.8662  0.8686   0.8618  0.8559   

                                              auc                       f1  \
feature                              seq complete  energy     seq complete   
data         model                                                           
HCV_WT       logistic_regression  0.9092   0.9463  0.9411  0.9501   0.6645   
             random_forest        0.9220   0.9010  0.8874  0.9581   0.6686   
             decision_tree        0.9023   0.7132  0.7211  0.8398   0.5824   
             svm                  0.9264   0.9413  0.9256  0.9593   0.8000   
             ann                  0.9204   0.9483  0.9308  0.9573   0.7824   
             PGCN                 0.9343   0.9559  0.9379  0.9547   0.8473   
HCV_A171T    logistic_regression  0.9538   0.9767  0.9736  0.9801   0.7416   
             random_forest        0.9410   0.9318  0.9208  0.9690   0.8078   
             decision_tree        0.9387   0.7132  0.6746  0.9129   0.5857   
             svm                  0.9567   0.9763  0.9

In [6]:
df_pivot.to_csv('MetricSummary_ML_PGCN_suppl.csv')